In [25]:
import pandas as pd
import os
import json

In [64]:
# df.sha.fillna("", inplace=True)
df = pd.read_csv("./CORD-19-research-challenge/metadata.csv")
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,8q5ondtn,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(72)90077-4
1,pzfd0e50,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90355-5
2,22bka3gi,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90356-7
3,zp9k1k3z,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(73)90176-9
4,cjuzul89,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-9343(85)90361-4


In [52]:
json_folder = "./CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/"

#get full text

def get_full_text(sha):
    if sha == "":
        return []
    filename = str(sha)+".json"
    file_path = os.path.join(json_folder, filename)
    
    with open(file_path) as json_file:
        data = json.load(json_file)
        full_text = []
        for section in ["abstract", "body_text"]:
            for paragraph in data[section]:
                full_text.append(paragraph["text"])
            
    return full_text 

In [53]:
df.sha.fillna("", inplace=True)
df['full_text'] = df.apply(lambda x: get_full_text(x.sha), axis=1)

FileNotFoundError: [Errno 2] No such file or directory: './CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/aecbc613ebdab36753235197ffb4f35734b5ca63.json'

In [70]:
# for x in df.sha[df['sha'].notnull()]:
#     get_full_text(x)

df.sha[df['sha'].notnull()].shape

(34283,)